In [1]:
#Please create a new folder named "all" to store all txt files of threads in the original folder.
from bs4 import BeautifulSoup, Tag
from collections import OrderedDict
from pathlib import Path
import os
import nltk
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from sklearn.metrics import classification_report
from graph_constructor import get_graph, get_digraph, get_graph_from_list, get_chronological_clustering_coef, get_chronological_pgrank
# from key_student_extractor import get_key_student
from preprocess import preprocess_text
import networkx as nx
import util

#html-css-javascript LgWwihnoEeWDtQoum3sFeQ
#python_database eQJvsjn9EeWJaxK5AT4frw
#python_network Y4DUPDpQEeWO-Qq6rEZAow
#python 7A1yFTaREeWWBQrVFXqd1w
#hybrid-mobile-development -gcU5xn4EeWwrBKfKrqlSQ
#machine-learning Gtv4Xb1-EeS-ViIACwYKVQ
#learning-how-to-learn GdeNrll1EeSROyIACtiVvg
#angular-js 52blABnqEeW9dA4X94-nLQ
#server-side-development ngZrURn5EeWwrBKfKrqlSQ
#web-frameworks ycQnChn3EeWDtQoum3sFeQ

In [2]:
def is_word(string):
    for char in string:
        if not char.isalpha():
            return False
    return True

def get_bit_list(length, index):
    
    bit_list = []
    for i in range(length):
        if not i == index:
            bit_list.append(0)
        else:
            bit_list.append(1)
    
    return bit_list

def tf_idf(docs, queries, tokenizer):
    """
    performs TF-IDF vectorization for documents and queries
    Parameters
        ----------
        docs : list
            list of documents
        queries : list
            list of queries
        tokenizer : custom tokenizer function
    Returns
    -------
    tfs : sparse array,
        tfidf vectors for documents. Each row corresponds to a document.
    tfs_query: sparse array,
        tfidf vectors for queries. Each row corresponds to a query.
    dictionary: list
        sorted dictionary
    """
    model = str.maketrans(dict.fromkeys(string.punctuation))
    processed_docs = [d.lower().translate(model) for d in docs]
    processed_queries = [d.lower().translate(model) for d in queries]
    tfidf = TfidfVectorizer(stop_words='english', tokenizer=tokenizer)
    tfs = tfidf.fit_transform(processed_docs)
    tfs_query = tfidf.transform(processed_queries)
    return tfs, tfs_query, tfidf

def tokenize_text(docs):
    """
    custom tokenization function given a list of documents
    Parameters
        ----------
        docs : string
            a document
    Returns
    -------
    stems : list
        list of tokens
    """

    text = ''
    for d in docs:
        text += '' + d
    stemmer = PorterStemmer()
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        if is_word(item):
            stems.append(stemmer.stem(item))
    return stems

def get_vectors(texts, queries):
    descriptions = []
    descriptions.append('')
    vec_docs, vec_queries, tfidf_model = tf_idf(texts, queries, tokenize_text)
    return vec_docs, vec_queries, tfidf_model

def graph_of_thread(thread_id):
    
    graph = nx.DiGraph()
    posts = thread_post_set_dic[thread_id]
    
    for each_post_id in posts:
        graph.add_edge(each_post_id, thread_id)
        comments = post_comment_set_dic[each_post_id]
        for each_comment_id in comments:
            graph.add_edge(each_comment_id, each_post_id)


    return graph


def is_gratitude_message(text):
    
    
    cleanr = re.compile('<.*?>')
    cleaned_text = re.sub(cleanr, '', text)
    
    # The message should not contain any questions
    if '?' in cleaned_text:
        return False
    
    if len(cleaned_text.split()) <= 15 and ('thank' in cleaned_text or 'Thank' in cleaned_text):
        return True
    
    else:
        return False

In [3]:
# testing the new function 

text_a = 'I love my dog'
text_b = 'My dog is the best in the world'
text_c = 'I love my cat'
text_d = 'The world cup is dogs'

training_texts = []
testing_texts = []

training_texts.append(text_a)
training_texts.append(text_b)
testing_texts.append(text_c)
testing_texts.append(text_d)

training_vectors, testing_vectors, model = get_vectors(training_texts, testing_texts)

training_vectors = training_vectors.toarray()
testing_vectors = testing_vectors.toarray()

print(training_vectors)
print(model.vocabulary_)
print(testing_vectors)

[[ 0.          0.57973867  0.81480247  0.        ]
 [ 0.6316672   0.44943642  0.          0.6316672 ]]
{'love': 2, 'dog': 1, 'best': 0, 'world': 3}
[[ 0.          0.          1.          0.        ]
 [ 0.          0.57973867  0.          0.81480247]]


In [4]:
database = input("please input the database name: ")

user_table = {}

conn = util.create_connection(database)
with conn:
    cur = conn.cursor() 
    cur.execute("SELECT id, user_title FROM user")
    rows = cur.fetchall()
    for each_user in rows:
        user_id = each_user[0]
        user_title = each_user[1]
        user_table[user_id] = user_title



please input the database name: courseraData.db


In [18]:
# list with the same data length

## texts
texts = []
training_texts = []
testing_texts = []

## ids
ids = []
training_ids = []
testing_ids = []

isreplied= []
id_to_index = {}

# for forum id alone
forum_id_list = []

# raw text dic
thread_text_dic = {}
post_text_dic = {}
comment_text_dic = {}

# container dic
forum_type_dic = {}
forum_graph_dic = {}
forum_digraph_dic = {}
thread_post_set_dic = {}
post_comment_set_dic = {}
thread_forum_dic = {}

# feature dic
num_of_post_dic = {}
num_of_comment_dic = {}
num_of_sen_dic = {}
num_of_comment_for_post_dic = {}
num_of_url_dic = {}
num_of_timeref_dic = {}
num_of_votes_dic = {}
is_replied = {}
is_instructor_replied = {}
starter_dic = {}

# truncation
thread_intervened_time_dic = {}
thread_posted_time_dic = {}

# threads that have gratitude messages
gratitude_message_dic = {}

# problem thread
replied_thread_without_reply_time = []

In [ ]:
course_id = input("please input your course_id: ")

input_method = input("please press 'a' for manual, 'b' for semi-auto scan, others for auto: ")
if input_method == 'a':
    num_forum = input("please input the number of forums: ")
    print("please input the forum ids in chronological order")
    for i in range(0, int(num_forum)):
        new_forum_id = input("please input the current forum id: ")
        forum_id_list.append(new_forum_id)
        forum_type_dic[new_forum_id] = i
        forum_graph_dic[new_forum_id] = get_graph(new_forum_id, database)
        forum_digraph_dic[new_forum_id] = get_digraph(new_forum_id, database)

elif input_method == 'b':
    path = 'gratitude_text/' + course_id + "/" + "forum.txt"
    file = open(path, "r")
    num_forum = int(file.readline())
    num_forum = input("please input the number of forums: ")
    for i in range (0, int(num_forum)):
        new_forum_id = file.readline()
        new_forum_id = new_forum_id.rstrip(os.linesep)
        forum_id_list.append(new_forum_id)
        forum_type_dic[new_forum_id] = i
        forum_graph_dic[new_forum_id] = get_graph(new_forum_id, database)
        forum_digraph_dic[new_forum_id] = get_digraph(new_forum_id, database)

else:
    path = 'gratitude_text/' + course_id + "/" + "forum.txt"
    file = open(path, "r")
    num_forum = int(file.readline())
    for i in range (0, num_forum):
        new_forum_id = file.readline()
        new_forum_id = new_forum_id.rstrip(os.linesep)
        forum_id_list.append(new_forum_id)
        forum_type_dic[new_forum_id] = i
        forum_graph_dic[new_forum_id] = get_graph(new_forum_id, database)
        forum_digraph_dic[new_forum_id] = get_digraph(new_forum_id, database)

middleInd = int(input("please input the training set scale: "))

limit_message = "WHERE courseid = \'"
limit_message += course_id
limit_message += "\'"
if int(num_forum) >= 1:
    limit_message += " AND (forumid = \'"
    limit_message += forum_id_list[0]
    limit_message += "\'"

for i in range(1, int(num_forum)):
    limit_message += " OR forumid = \'"
    limit_message += forum_id_list[i]
    limit_message += "\'"

if int(num_forum) >= 1:
    limit_message += ")"
    
print("done preprocessing!")


please input your course_id: ngZrURn5EeWwrBKfKrqlSQ
please press 'a' for manual, 'b' for semi-auto scan, others for auto: b
please input the number of forums: 4


In [13]:
# extract data from db.
ids = []
training_ids = []
testing_ids = []

changed_thread_ids = []

conn = util.create_connection(database)
with conn:
    cur = conn.cursor()
    thread_message = 'SELECT id, title, inst_replied, starter, forumid, posted_time FROM thread '
    thread_message += limit_message
    thread_message += ' ORDER BY posted_time'
    print(thread_message)
    cur.execute(thread_message)
    threads = cur.fetchall()
    for each_thread in threads:
        
        # All the info from this select operation
        thread_id = each_thread[0]
        thread_title = each_thread[1]
        thread_intervention = each_thread[2]
        thread_starter = each_thread[3]
        thread_forum_id = each_thread[4]
        thread_time = each_thread[5]
        
        # Initialize intervention time as -1
        thread_intervened_time_dic[thread_id] = -1
        if(thread_starter in user_table and (user_table[thread_starter] == 'Instructor' or user_table[thread_starter] == 'Staff')):           
            continue
        
        # Memorise whether it has a instructor intervention (as compared to gratitude intervwntion)
        is_instructor_replied[thread_id] = thread_intervention
        
        # Detect the earliest intervention.
        post_message = "SELECT user, post_time, post_text FROM post WHERE thread_id = \""
        post_message += thread_id
        post_message += "\" ORDER BY post_time"
        cur.execute(post_message)
        all_posts = cur.fetchall()
        for each_post in all_posts:

            # parameters
            poster = each_post[0]
            post_time = each_post[1]
            post_text = each_post[2]

            if poster in user_table and (user_table[poster] == 'Instructor' or user_table[poster] == 'Staff' or is_gratitude_message(post_text)) and thread_intervened_time_dic[thread_id] == -1:
                thread_intervened_time_dic[thread_id] = post_time

            if is_gratitude_message(post_text) and forum_type_dic[thread_forum_id] <= middleInd:
                print('grat intervention post: ', post_text)
                changed_thread_ids.append(thread_id)
                thread_intervention = 1

        comment_message = "SELECT user, post_time, comment_text, id FROM comment WHERE thread_id = \""
        comment_message += thread_id
        comment_message += "\" ORDER BY post_time"
        cur.execute(comment_message)
        all_comments = cur.fetchall()
        for each_comment in all_comments:

            # parameters
            commenter = each_comment[0]
            comment_time = each_comment[1]
            comment_text = each_comment[2]
            comment_id = each_comment[3]

            if commenter in user_table and (user_table[commenter] == 'Instructor' or user_table[commenter] == 'Staff' or is_gratitude_message(comment_text)):
                if thread_intervened_time_dic[thread_id] == -1 or comment_time < thread_intervened_time_dic[thread_id]:

                    thread_intervened_time_dic[thread_id] = comment_time

            if is_gratitude_message(comment_text) and forum_type_dic[thread_forum_id] <= middleInd:
                if thread_intervened_time_dic[thread_id] == -1 or comment_time < thread_intervened_time_dic[thread_id]:

                    print('grat intervention comment: ', comment_text)
                    changed_thread_ids.append(thread_id)
                    thread_intervention = 1

        if thread_intervened_time_dic[thread_id] == -1:
            replied_thread_without_reply_time.append(thread_id)
        
        # Memorize all the info in their respective dics
        ids.append(thread_id)
        thread_posted_time_dic[thread_id] = thread_time
        thread_forum_dic[thread_id] = thread_forum_id
        is_replied[thread_id] = thread_intervention
        thread_post_set_dic[thread_id] = []
        num_of_post_dic[thread_id] = 0
        num_of_comment_dic[thread_id] = 0
        num_of_votes_dic[thread_id] = 0
        starter_dic[thread_id] = thread_starter
        isreplied.append(thread_intervention)
        thread_text_dic[thread_id] = thread_title

    # extract all the posts
    post_message = 'SELECT id, thread_id, votes, user, post_text, post_time FROM post '
    post_message += limit_message
    post_message += ' ORDER BY post_time'
    cur.execute(post_message)
    rows = cur.fetchall()
    for each_post in rows:
        
        # All the info from this select operation
        post_id = each_post[0]
        post_thread_id = each_post[1]
        post_vote = each_post[2]
        poster = each_post[3]
        post_text = each_post[4]
        post_time = each_post[5]
        
        # If the home thread is intervened, continue.
        if post_thread_id in is_replied and is_replied[post_thread_id] and thread_intervened_time_dic[post_thread_id] != -1 and post_time >= thread_intervened_time_dic[post_thread_id]:
            continue
        
        # initialize all the features dic for posts
        post_comment_set_dic[post_id] = []
        num_of_comment_for_post_dic[post_id] = 0 
        post_text_dic[post_id] = post_text

        # if the thread is not initialized by the instructor, add thread level features
        if post_thread_id in num_of_votes_dic:
            num_of_votes_dic[post_thread_id] += post_vote
            thread_post_set_dic[post_thread_id].append(post_id)
            num_of_post_dic[post_thread_id] += 1


    comment_message = 'SELECT id, thread_id, post_id, user, comment_text, post_time FROM comment '
    comment_message += limit_message
    comment_message += ' ORDER BY post_time'

    cur.execute(comment_message)
    print(comment_message)
    rows = cur.fetchall()
    for each_comment in rows:
        
        # All the info from this select operation
        comment_id = each_comment[0]
        comment_thread_id = each_comment[1]
        comment_post_id = each_comment[2]
        commenter = each_comment[3]
        comment_text = each_comment[4]
        comment_time = each_comment[5]

        # if after intervention, truncate
        if comment_thread_id in is_replied and is_replied[comment_thread_id] and thread_intervened_time_dic[post_thread_id] != -1 and comment_time >= thread_intervened_time_dic[comment_thread_id]:
            continue
 
        # features
        comment_text_dic[comment_id] = each_comment[4]

        if comment_thread_id in num_of_comment_dic:
            num_of_comment_dic[comment_thread_id] +=  1
            
        if comment_post_id in num_of_comment_for_post_dic:
            num_of_comment_for_post_dic[comment_post_id] += 1
            post_comment_set_dic[comment_post_id].append(comment_id)
            
print(len(ids))

SELECT id, title, inst_replied, starter, forumid, posted_time FROM thread WHERE courseid = 'GdeNrll1EeSROyIACtiVvg' AND (forumid = 'GdeNrll1EeSROyIACtiVvg' OR forumid = 'm4nU0ygeEeaZ8Apto8QB_w' OR forumid = 'm5T43ygeEea7jBLLHPwd0w') ORDER BY posted_time
grat intervention post:  <co-content><text>Ah oh... lol</text><text>Such a cute professor! I really enjoyed the learning. </text><text>Thank you so much:)</text></co-content>
grat intervention post:  <co-content><text>Thanks Robin, that's a really interesting article.</text></co-content>
grat intervention post:  <co-content><text>Thanks! </text></co-content>
grat intervention post:  <co-content><text>Thanks Robin - I'll pass it on. As you suspected, it's not supposed to happen.</text></co-content>
grat intervention post:  <co-content><text>thanks</text></co-content>
grat intervention post:  <co-content><text>Thank You 😊😊</text></co-content>
SELECT id, thread_id, post_id, user, comment_text, post_time FROM comment WHERE courseid = 'GdeNr

In [14]:
# construct the id to index dictionary and split the training ids and testing ids
index = 0
training_ids = []
testing_ids = []

# ensure add from scratch
texts = []
training_texts = []
testing_texts = []

# storing the ids for the training set and test set
for oneid in ids:
    id_to_index[oneid] = index
    if forum_type_dic[thread_forum_dic[oneid]] <= middleInd:
        training_ids.append(oneid)
    else:
        testing_ids.append(oneid)

    index += 1

# process text
for oneid in ids:
    
    filename = 'text/' + course_id + "/" + str(oneid) + '.txt'
    f = open(filename, 'r', errors='ignore')
    content = f.read()
    
    # features
    num_url = content.count('a href')             
    num_timeref = content.count('<TIMEREF>')
    sen_list = re.split(r'[.!?]+', content)
    num_sen = len(sen_list)
    
    # store features
    num_of_url_dic[oneid] = num_url
    num_of_timeref_dic[oneid] = num_timeref
    num_of_sen_dic[oneid] = num_sen

    soup = BeautifulSoup(content, "lxml")

    for tag in soup.find_all('code'):
        tag.replaceWith('')

    for tag in soup.find_all('a'):
        tag.replaceWith('')

    content = soup.get_text()
    content = re.sub(r'^https?:\/\/.*[\r\n]*', '', content, flags=re.MULTILINE)

    if oneid in training_ids:
        training_texts.append(content)
    else:
        testing_texts.append(content)
    texts.append(content)

# get the training vector and test vector
training_vectors, testing_vectors, tfidf_model = get_vectors(training_texts, testing_texts)
tmp_vectors = []
current_training_index = 0
current_testing_index = 0

training_vectors = training_vectors.toarray()
testing_vectors = testing_vectors.toarray()

for oneid in ids:
    if oneid in training_ids:
        tmp_vectors.append(training_vectors[current_training_index])
        current_training_index += 1
    else:
        tmp_vectors.append(testing_vectors[current_testing_index])
        current_testing_index += 1

if current_training_index != len(training_vectors) or current_testing_index != len(testing_vectors):
    print("Fatal error!")



data_length = len(ids)

In [15]:
training_vectors = []
test_vectors = []
training_result = []
test_result = []

# normalize the term array
x = np.asarray(tmp_vectors)
#x = sklearn.preprocessing.normalize(x, norm='l2', axis=0, copy=True, return_norm=False)

### EDM 15

# additional vectors to be appended later
additional_vectors = []

for oneid in ids:
    
    # get the index from the first to the last in the id list
    index = id_to_index[oneid]
    origin_list = x[index].tolist()
    
    # temporary list storing the newly added features    
    forum_index = forum_type_dic[thread_forum_dic[oneid]]
    forum_bit_list = get_bit_list(int(num_forum), int(forum_index))
    li = forum_bit_list
    
    numpost = num_of_post_dic[oneid]
    li.append(numpost)
    numcomment = num_of_comment_dic[oneid]
    li.append(numcomment)
    li.append(numpost + numcomment)
    
    # will add the average # of comments per post
    summ = 0
    for postid in thread_post_set_dic[oneid]:
        summ = summ + num_of_comment_for_post_dic[postid]
     
    if len(thread_post_set_dic[oneid]) == 0:
        avr = 0
    else:
        avr = float(summ / len(thread_post_set_dic[oneid]))

    li.append(avr)
    numurl = num_of_url_dic[oneid]
    li.append(numurl)
    numsen = num_of_sen_dic[oneid]
    li.append(numsen)
    numvotes = num_of_votes_dic[oneid]
    li.append(numvotes)

    numtimeref = num_of_timeref_dic[oneid]
    additional_vectors.append(li)

# normalize the new vectors (max_min norm)
additional_vectors = np.asarray(additional_vectors)
scaler = sklearn.preprocessing.MinMaxScaler()
scaler.fit(additional_vectors)
additional_vectors = scaler.transform(additional_vectors)

# append new feature vectors to the term vectors
x_edm = np.append(x, additional_vectors, axis = 1)
y_edm = np.asarray(isreplied)

print(x_edm)

for oneid in ids:
    
    # get the index from the first to the last in the id list
    index = id_to_index[oneid]
    origin_list = x_edm[index].tolist()
    
    # split the vectors into training and test set
    forum_id = thread_forum_dic[oneid]
    forum_number = forum_type_dic[forum_id]
    
    if forum_number <= middleInd:
        training_vectors.append(origin_list)
        training_result.append(isreplied[index])
    else:
        test_vectors.append(origin_list)
        test_result.append(isreplied[index])

LogReg = LogisticRegression(class_weight = 'balanced')
LogReg.fit(training_vectors, training_result)
pred_result = LogReg.predict(test_vectors)
with open('EDM.txt', 'w') as f:
    print(classification_report(test_result, pred_result), file = f)
print(classification_report(test_result, pred_result))

for i in range(len(test_result)):
    if test_result[i] == 1 and pred_result[i] == 0:
        print('did not extract')
        print(testing_ids[i])
        print(texts[id_to_index[testing_ids[i]]])
        print('\n')
    elif test_result[i] == 0 and pred_result[i] == 1:
        print('rubbish extracted')
        print(testing_ids[i])
        print(texts[id_to_index[testing_ids[i]]])
        print('\n')

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.57142857  1.          0.02678571]
 [ 0.          0.          0.         ...,  0.          0.04545455
   0.05357143]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          1.        ]
 [ 0.          0.          0.         ...,  0.          0.04545455  0.        ]
 [ 0.          0.          0.         ...,  0.          0.06818182  0.        ]]
             precision    recall  f1-score   support

          0       0.92      0.96      0.94        48
          1       0.50      0.33      0.40         6

avg / total       0.87      0.89      0.88        54

did not extract
GdeNrll1EeSROyIACtiVvg~rgWlD4O2EeWwyg6oWwzVsw
technical issue
the video by the name of summing up memory is not working



did not extract
GdeNrll1EeSROyIACtiVvg~-VECyI6iEeW7vBKNDxH-cw
the cue that launches into procrastination
In my case, I think the cue that launches me into p

In [16]:
# track the number of positive instances in the training set
pos_result = 0
neg_result = 0

for bit in training_result:
    if not bit == 0:
        pos_result += 1
    else:
        neg_result += 1
        
print(pos_result)
print(neg_result)

print(len(training_result))

13
34
47


In [17]:
for thread_id in changed_thread_ids:
    
    print(is_instructor_replied[thread_id])
    print


0
1
1
1
1
1


In [13]:
for i in range(len(test_result)):
    if test_result[i] == 1 and pred_result[i] == 0:
        print('did not extract')
        print(testing_ids[i])
        print(texts[id_to_index[testing_ids[i]]])
        print('\n')
    elif test_result[i] == 0 and pred_result[i] == 1:
        print('rubbish extracted')
        print(testing_ids[i])
        print(texts[id_to_index[testing_ids[i]]])
        print('\n')

did not extract
ycQnChn3EeWDtQoum3sFeQ~MRtFO3c5EeWdPxJ-h_8T7w
Dear all, use external resources (Bootstrap CDN) for CSS and JS 

While reviewing my classmates I realized that it's hard to examine HTML pages because of missed styles.I suggest using external resources for CSS and JS as professor Jogesh mentioned before (Bootstrap CDN - . Or simply host files on your Google drive - OR you can use my compilation-------- before </head> --------<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css">    <link rel="stylesheet" href="https://googledrive.com/host/0BwxB5DwSCJhrVnRZWi04SjNsOEE">    <link rel="stylesheet" href="https://d396qusza40orc.cloudfront.net/phoenixassets/web-frameworks/bootstrap-social.css"> <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.4.0/css/font-awesome.min.css">AND-------- before </body> --------<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.3/jquery.min.js"></script>    <script s




rubbish extracted
ycQnChn3EeWDtQoum3sFeQ~PBHomYS9EeWA9w4ipzCZ2Q
Checkbox in lightly unaligned horizontally with the "remember me" text

My checkbox in lightly unaligned horizontally with the "remember me" text.How can i fix this?
That is normal, and it looks the same at my end too.
Thank´s for your answer but in the video it looks pretty much align. 
That's because he's using Chrome for Mac OS instead of Chrome for Windows.Mac uses a different set of visuals than Windows and so the radio and checkbox buttons look and align differently.
Using chrome for linux the alignment is good.
I see is a windows misalignment...
Is Bootstrap not supposed to have some CSS reset in order to avoid this compatibility problems?
I also found that by changing the class of the outer div (visible on line 79 in the video between 6:10 and 6:30) from checkbox to checkbox-inline seems to fix this issue.Also here I have a question in mind, why not simply to get ride of all unnecessary divs.For example keeping 